# Airbags e outras influências em acidentes fatais 
   ### por Arthur Um, Lucas Lai, e Fernando Chiu


Fonte: National Highway Traffic Safety Administration - uma agência do poder executivo do governo dos EUA, parte do Departamento de Transporte. 
   - NASSCDS - National Automotive Sampling System e o seu Crashworthiness Data System
     - casos selecionados a partir de relatórios de acidentes(batidas) de carro da polícia 
     - melhorias no design de veículos

‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌
‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌‌
Dados de 1997-2002 sobre acidentes de carro nos EUA em quais ocorreram algum evento prejudicial às vítimas ou suas propriedades, e em que pelo menos um dos veículos envolvidos teve que ser tirado a guincho. A coleção de dados foi realizada através de um esquema de amostragem multiestágio: a população foi dividida em grupos(clusters) e aleatoriamente selecionadas em etapas posteriores. Os dados estão limitados a apenas os passageiros que sentaram nos bancos dianteiros do carro, entre outras limitações.  


In [1]:
df<-read.csv("Car accidents.csv")
str(df)

'data.frame':	26217 obs. of  16 variables:
 $ X          : int  1 2 3 4 5 6 7 8 9 10 ...
 $ dvcat      : Factor w/ 5 levels "1-9km/h","10-24",..: 3 2 2 3 3 4 5 5 2 2 ...
 $ weight     : num  25.1 25.1 32.4 495.4 25.1 ...
 $ dead       : Factor w/ 2 levels "alive","dead": 1 1 1 1 1 1 1 2 1 1 ...
 $ airbag     : Factor w/ 2 levels "airbag","none": 2 1 2 1 2 2 2 2 2 2 ...
 $ seatbelt   : Factor w/ 2 levels "belted","none": 1 1 2 1 1 1 1 2 1 1 ...
 $ frontal    : int  1 1 1 1 1 1 1 1 0 1 ...
 $ sex        : Factor w/ 2 levels "f","m": 1 1 1 1 1 1 2 2 2 1 ...
 $ ageOFocc   : int  26 72 69 53 32 22 22 32 40 18 ...
 $ yearacc    : int  1997 1997 1997 1997 1997 1997 1997 1997 1997 1997 ...
 $ yearVeh    : int  1990 1995 1988 1995 1988 1985 1984 1987 1984 1987 ...
 $ abcat      : Factor w/ 3 levels "deploy","nodeploy",..: 3 1 3 1 3 3 3 3 3 3 ...
 $ occRole    : Factor w/ 2 levels "driver","pass": 1 1 1 1 1 1 1 1 1 1 ...
 $ deploy     : int  0 1 0 1 0 0 0 0 0 0 ...
 $ injSeverity: int  3 1 4 1 3

- X - numeração das observações
- dvcat - velocidade do impacto estimado (1-9km/h, 10-24, 25-39, 40-54, 55+). **Quantitativa Discreta** 
- weight - os pesos das observações, porém com incerteza na precisão dos mesmos. **Quantitativa Contínua**
- dead - se a vítima observada faleceu ou não (alive, dead). **Qualitativa Dicotômica**
- airbag - se o veículo observado possuía airbag ou não (none, airbag). **Qualitativa Dicotômica**
- seatbelt - se a vítima observada estava com cinto ou não (none, belted). **Qualitativa Dicotômica**
- frontal - se o impacto da observação foi frontal ou não (numérico;  0 = não frontal, 1=frontal). **Qualitativa Dicotômica**
- sex - sexo da vítima observada (f,m). **Qualitativa Dicotômica**
- ageOFocc - idade do ocupante da observação em anos. **Quantitativa Contínua** 
- yearacc - ano em que o acidente ocorreu. **Quantitativa Discreta**
- yearVeh - ano do modelo do carro. **Quantitativa Contínua**
- abcat - um ou mais airbags foram ativados(seja motorista ou passageiro)? (deploy, nodeploy, unavail). **Qualitativa Nominal**
- occRole - papel do ocupante da observação (driver,pass - motorista ou passageiro). **Qualitativa Dicotômica** 
- deploy - mesmo que abcat, porém numérico (0 caso não houve airbag, 1 caso houve). **Qualitativa Dicotômica** 
- injSeverity - severidade do ferimento da vítima observada, numérico; 0:nenhum, 1:possivelmente ferido, 2:incapacitação, 3:incapacitado, 4:morto; 5: desconhecido, 6:morte precedente **Qualitativa Ordinal**
- caseid - tipo: character; populations sampling unit + case number + vehicle number. Podemos utilizá-lo para identificar o veículo como único em cada ano. 


In [2]:
# Tiramos as colunas 'X' e 'caseid'
df$X<-NULL
df$caseid<-NULL
# Contamos o no. de na values em cada coluna
colSums(is.na(df))

dvcat      weight        dead      airbag    seatbelt     frontal 
          0           0           0           0           0           0 
        sex    ageOFocc     yearacc     yearVeh       abcat     occRole 
          0           0           0           1           0           0 
     deploy injSeverity 
          0         153

In [3]:
df<-na.omit(df) # Excluimos os valores na
table(is.na(df))
message("No. de obs.:")
nrow(df) # Antes era 26217 


 FALSE 
364882 

No. de obs.:


[1] 26063

     dvcat           weight            dead          airbag        seatbelt    
 1-9km/h:  676   Min.   :    0.00   alive:24883   airbag:14336   belted:18465  
 10-24  :12766   1st Qu.:   32.38   dead : 1180   none  :11727   none  : 7598  
 25-39  : 8165   Median :   86.99                                              
 40-54  : 2965   Mean   :  462.48                                              
 55+    : 1491   3rd Qu.:  363.35                                              
                 Max.   :57871.59                                              
    frontal       sex          ageOFocc        yearacc        yearVeh    
 Min.   :0.0000   f:12178   Min.   :16.00   Min.   :1997   Min.   :1953  
 1st Qu.:0.0000   m:13885   1st Qu.:22.00   1st Qu.:1998   1st Qu.:1989  
 Median :1.0000             Median :33.00   Median :2000   Median :1994  
 Mean   :0.6436             Mean   :37.22   Mean   :2000   Mean   :1993  
 3rd Qu.:1.0000             3rd Qu.:48.00   3rd Qu.:2001   3rd Qu.:199

 Buscamos investigar os fatores que influenciam nos casos de morte, e na gravidade dos ferimentos. Por exemplo a relacao entre airbag e cinto de seguranca e a gravidade dos ferimentos da vitima. 
